# Constructing our Minimum-Covariance Portfolios

We will be constructing our portfolios by selecting the assets which have the lowest correlations between each other. We will rebalance semi-annually based on these selections, and at each rebalance, ensure that our portfolio is split among the two geographies 50/50 

In [17]:
import numpy as np
import pandas as pd

In [18]:
# Remove 1st line since they are pct changes, therefore have
# NaNs in the first row
rets = pd.read_csv("returns.csv")[1:]
rets.head()

,Date,URTH,VEA,SPDW,IDEV,GSIE,IVLU,IGOV,GWX,IVV,...,DVYE,HYEM,AFK,CNY=X,BRL=X,ZAR=X,RUB=X,EUR=X,JPY=X,MXN=X
1,2011-01-04,NaN,NaN,-0.005084,NaN,NaN,NaN,-0.004333,-0.002978,-0.000674,...,NaN,NaN,0.000777,-0.001760,0.019189,0.006326,0.0,0.000134,0.006743,-0.000384
2,2011-01-05,NaN,NaN,0.000000,NaN,NaN,NaN,-0.015666,-0.003484,0.005105,...,NaN,NaN,0.012422,0.002584,0.012754,0.003853,0.0,0.002671,0.004506,-0.004249
3,2011-01-06,NaN,NaN,-0.008687,NaN,NaN,NaN,-0.006189,-0.009491,-0.001821,...,NaN,NaN,-0.010736,-0.002956,0.004498,0.008670,0.0,0.012522,0.014030,0.003119
4,2011-01-07,NaN,NaN,-0.005155,NaN,NaN,NaN,-0.006228,-0.001009,-0.001824,...,NaN,NaN,-0.004651,0.005929,0.006507,0.012983,0.0,0.013551,0.001334,0.000573
5,2011-01-10,NaN,NaN,-0.004145,NaN,NaN,NaN,0.005595,-0.002019,-0.001058,...,NaN,NaN,0.003115,0.000151,-0.029895,-0.000295,0.0,0.006620,-0.003241,-0.001063


In [19]:
# Reindex to date
rets.index = pd.to_datetime(rets['Date'])
del rets['Date']
rets.head()

,URTH,VEA,SPDW,IDEV,GSIE,IVLU,IGOV,GWX,IVV,VTV,...,DVYE,HYEM,AFK,CNY=X,BRL=X,ZAR=X,RUB=X,EUR=X,JPY=X,MXN=X
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-04,NaN,NaN,-0.005084,NaN,NaN,NaN,-0.004333,-0.002978,-0.000674,0.001703,...,NaN,NaN,0.000777,-0.001760,0.019189,0.006326,0.0,0.000134,0.006743,-0.000384
2011-01-05,NaN,NaN,0.000000,NaN,NaN,NaN,-0.015666,-0.003484,0.005105,0.004372,...,NaN,NaN,0.012422,0.002584,0.012754,0.003853,0.0,0.002671,0.004506,-0.004249
2011-01-06,NaN,NaN,-0.008687,NaN,NaN,NaN,-0.006189,-0.009491,-0.001821,-0.002660,...,NaN,NaN,-0.010736,-0.002956,0.004498,0.008670,0.0,0.012522,0.014030,0.003119
2011-01-07,NaN,NaN,-0.005155,NaN,NaN,NaN,-0.006228,-0.001009,-0.001824,-0.002425,...,NaN,NaN,-0.004651,0.005929,0.006507,0.012983,0.0,0.013551,0.001334,0.000573
2011-01-10,NaN,NaN,-0.004145,NaN,NaN,NaN,0.005595,-0.002019,-0.001058,-0.002674,...,NaN,NaN,0.003115,0.000151,-0.029895,-0.000295,0.0,0.006620,-0.003241,-0.001063


In [20]:
dates_to_split = pd.date_range(rets.index[0], rets.index[-1], freq='6M')
dates_to_split

DatetimeIndex(['2011-01-31', '2011-07-31', '2012-01-31', '2012-07-31',
               '2013-01-31', '2013-07-31', '2014-01-31', '2014-07-31',
               '2015-01-31', '2015-07-31', '2016-01-31', '2016-07-31',
               '2017-01-31', '2017-07-31', '2018-01-31', '2018-07-31',
               '2019-01-31', '2019-07-31', '2020-01-31', '2020-07-31',
               '2021-01-31'],
              dtype='datetime64[ns]', freq='6M')

In [22]:
# Split on these dates
semiannual = {}

for i in range(len(dates_to_split)-1):
    #print(dates_to_split[i+1])
    semiannual[i] = rets[dates_to_split[i]:dates_to_split[i+1]]

In [25]:
# Sanity check
print(semiannual[0].index[0],semiannual[0].index[-1])

2011-01-31 00:00:00 2011-07-29 00:00:00


In [26]:
# Sanity check
print(semiannual[1].index[0],semiannual[1].index[-1])

2011-08-01 00:00:00 2012-01-31 00:00:00


In [29]:
# Sanity check
print(semiannual[19].index[0],semiannual[19].index[-1])

2020-07-31 00:00:00 2021-01-29 00:00:00


In [34]:
# For each group of 6mo, find the assets with lowest covariance

semiannual[0].corr()

,URTH,VEA,SPDW,IDEV,GSIE,IVLU,IGOV,GWX,IVV,VTV,...,DVYE,HYEM,AFK,CNY=X,BRL=X,ZAR=X,RUB=X,EUR=X,JPY=X,MXN=X
URTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VEA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SPDW,NaN,NaN,1.000000,NaN,NaN,NaN,0.489650,0.945869,0.883840,0.881518,...,NaN,NaN,0.779999,0.172785,-0.049264,0.010936,0.075694,-0.060653,0.005977,-0.762590
IDEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZAR=X,NaN,NaN,0.010936,NaN,NaN,NaN,-0.027213,-0.006197,0.034154,0.028845,...,NaN,NaN,0.000503,0.038800,0.367151,1.000000,0.276179,0.661781,-0.206965,-0.044212
RUB=X,NaN,NaN,0.075694,NaN,NaN,NaN,0.028720,0.037274,0.098897,0.071238,...,NaN,NaN,0.087330,0.493641,0.172991,0.276179,1.000000,0.195078,-0.018799,-0.037174
EUR=X,NaN,NaN,-0.060653,NaN,NaN,NaN,-0.130854,-0.073639,-0.009562,-0.002947,...,NaN,NaN,-0.033115,-0.033623,0.253644,0.661781,0.195078,1.000000,-0.036988,-0.028294
JPY=X,NaN,NaN,0.005977,NaN,NaN,NaN,-0.152979,0.032209,0.038298,0.034331,...,NaN,NaN,-0.075376,-0.002890,-0.107355,-0.206965,-0.018799,-0.036988,1.000000,0.097911
